# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [2]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(6876, 69)


(6876, 69)

In [3]:
# 2) seu código aqui
print(sinasc.isnull().sum())


ORIGEM         0
CODESTAB      24
CODMUNNASC     0
LOCNASC        0
IDADEMAE       0
              ..
munResUf       1
munResLat      2
munResLon      2
munResAlt      2
munResArea     2
Length: 69, dtype: int64


In [4]:
# 3) seu código aqui
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_interesse = sinasc[colunas_interesse]
print(sinasc_interesse.isnull().sum())

LOCNASC         0
IDADEMAE        0
ESTCIVMAE      83
ESCMAE         51
QTDFILVIVO    223
GESTACAO      340
GRAVIDEZ       15
CONSULTAS       0
APGAR5         18
dtype: int64


In [6]:
# 4) seu código aqui
sinasc_interesse = sinasc_interesse.dropna(subset=['APGAR5'])
print(sinasc_interesse.shape)
print(sinasc_interesse.isnull().sum())

(6858, 9)
LOCNASC         0
IDADEMAE        0
ESTCIVMAE      81
ESCMAE         51
QTDFILVIVO    223
GESTACAO      339
GRAVIDEZ       15
CONSULTAS       0
APGAR5          0
dtype: int64


In [7]:
# 5) seu código aqui
sinasc_interesse['ESTCIVMAE'] = sinasc_interesse['ESTCIVMAE'].fillna(9)
sinasc_interesse['CONSULTAS'] = sinasc_interesse['CONSULTAS'].fillna(9)

In [8]:
# 6) Seu código aqui
sinasc_interesse['QTDFILVIVO'] = sinasc_interesse['QTDFILVIVO'].fillna(0)

In [9]:
# 7) seu código aqui
sinasc_interesse['IDADEMAE'] = sinasc_interesse['IDADEMAE'].fillna(9)
sinasc_interesse['ESCMAE'] = sinasc_interesse['ESCMAE'].fillna(9)
sinasc_interesse['GESTACAO'] = sinasc_interesse['GESTACAO'].fillna(9)
sinasc_interesse['GRAVIDEZ'] = sinasc_interesse['GRAVIDEZ'].fillna(9)

In [11]:
# 8) seu código aqui
def categorizar_apgar(apgar):
  if apgar >= 8:
    return 'normal'
  elif apgar >= 6:
    return 'leve'
  elif apgar >= 4:
    return 'moderada'
  else:
    return 'severa'

sinasc_interesse['apgar_cat'] = sinasc_interesse['APGAR5'].apply(categorizar_apgar)
print(sinasc_interesse['apgar_cat'].value_counts())

normal      6727
leve          86
severa        26
moderada      19
Name: apgar_cat, dtype: int64


In [12]:
# 9) seu código aqui
sinasc_interesse.rename(columns={'LOCNASC': 'loc_nasc', 'IDADEMAE': 'idade_mae', 'ESTCIVMAE': 'est_civ_mae', 'ESCMAE': 'esc_mae',
'QTDFILVIVO': 'qtd_fil_vivo', 'GESTACAO': 'gestacao', 'GRAVIDEZ': 'gravidez', 'CONSULTAS': 'consultas',
'APGAR5': 'apgar5'}, inplace=True)

print(sinasc_interesse.columns)

Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar5', 'apgar_cat'],
      dtype='object')
